# purpose: to edit the metadata of saildrone files, as non-intrusively as possible

this code:\
imports relevant packages\
defines a tictoc function\
specifies the mission and year. this code must be ran mission by mission and year by year\
reads in appropriate dictionaries\
presents the giant for loop that edits the metadata of each applicable saildrone file\
...this loop is different for files that already have keywords that we want to keep versus those that do not\
presents a small set of checks to make sure the code is running correctly

In [1]:
import netCDF4
import h5py
#import h5netcdf.legacyapi as netCDF4
import numpy as np
import xarray as xr
import os
import numpy as np
from nco import Nco
from nco.custom import Atted, Rename
nco = Nco()
from glob import glob
import csv
import datetime
os.environ['HDF5_USE_FILE_LOCKING']='FALSE'


##### tictoc

In [2]:
def tic():
    #Homemade version of matlab tic and toc functions
    import time
    global startTime_for_tictoc
    startTime_for_tictoc = time.time()

def toc():
    import time
    if 'startTime_for_tictoc' in globals():
        print ("Elapsed time is " + str(time.time() - startTime_for_tictoc) + " seconds.")
    else:
        print ("Toc: start time not set")

### read in dicts

In [3]:
mission_name = 'tpos'#state the project
year = '2018'#state the year

In [4]:
filedir = '/home/users/koukel/saildrone_metawork/'#the directory all metadata csvs are in
filenam_static = 'saildronemeta_static.csv'#the csv for the static metadata
filenam_untouched = 'saildronemeta_untouched.csv'#the csv for the changing metadata that are not edited
filenam_changing = 'mission_csvs/saildronemeta_' + mission_name + '_' + year + '_changing.csv'#the csv for the particular mission and year
filenam_wmo = 'saildronemeta_wmo_ids.csv'#the csv with the wmo_ids
filenam_proj = 'saildronemeta_projectnames_missionnames.csv'#the csv with project_names and fancy Mission Names
filenam_inst = 'saildronemeta_institutionnames.csv'

reader_static = csv.reader(open(filedir + filenam_static))
reader_untouched = csv.reader(open(filedir + filenam_untouched))
reader_changing = csv.reader(open(filedir + filenam_changing))
reader_wmo = csv.reader(open(filedir + filenam_wmo))
reader_projnames = csv.reader(open(filedir + filenam_proj))
reader_instnames = csv.reader(open(filedir + filenam_inst))

In [5]:
static_metadata = {}
for k, v in reader_static:
    static_metadata[k] = v

In [6]:
static_metadata

{'_NCProperties': 'version=2,netcdf=4.6.3,hdf5=1.10.2',
 'cdm_data_type': 'Trajectory',
 'cdm_trajectory_variables': 'trajectory',
 'Conventions': 'CF-1.6, ACDD-1.3, COARDS',
 'creator_email': 'support@saildrone.com, oar.pmel.data@noaa.gov',
 'creator_type': 'Institution',
 'description': 'Saildrone NetCDF Format',
 'featureType': 'Trajectory',
 'geospatial_lat_units': 'degrees_north',
 'geospatial_lon_units': 'degrees_east',
 'keywords_vocabulary': 'GCMD Science Keywords',
 'license': 'These data were produced by NOAA and are not subject to copyright protection in the United States. NOAA waives any potential copyright and related rights in these data worldwide through the Creative Commons Zero 1.0 Universal Public Domain Dedication (CC0-1.0).',
 'naming_authority': 'com.saildrone',
 'ncei_template_version': 'NCEI_NetCDF_Trajectory_Template_v2.0',
 'netcdf_version': '4.6.3',
 'nodc_template_version': 'NODC_NetCDF_Trajectory_Template_v2.0',
 'platform': 'Saildrone',
 'publisher_email': 

In [7]:
untouched_metadata = {}
for k, v, j in reader_untouched:#j is third column but not read in
    untouched_metadata[k] = v

In [8]:
untouched_metadata

{'data_mode': '',
 'date_created': '',
 'drone_id': '',
 'id': '',
 'geospatial_lat_max': '',
 'geospatial_lat_min': '',
 'geospatial_lon_max': '',
 'geospatial_lon_min': '',
 'time_coverage_end': '',
 'time_coverage_start': ''}

In [9]:
geospatial_metadata = {'geospatial_lat_max':untouched_metadata['geospatial_lat_max'],'geospatial_lat_min':untouched_metadata['geospatial_lat_min'],
                       'geospatial_lon_max':untouched_metadata['geospatial_lon_max'],'geospatial_lon_min':untouched_metadata['geospatial_lon_min'],
                      }

In [10]:
geospatial_metadata

{'geospatial_lat_max': '',
 'geospatial_lat_min': '',
 'geospatial_lon_max': '',
 'geospatial_lon_min': ''}

In [11]:
changing_metadata = {}
for k, v, j in reader_changing:
    changing_metadata[k] = v

In [12]:
changing_metadata

{'acknowledgement': 'If you use these data in publications or presentations, please acknowledge OCS at NOAA PMEL. Also, we would appreciate receiving a preprint and/or reprint of presentations and publications utilizing the data for inclusion in our bibliography. Relevant publications should be sent to: NOAA/PMEL OCS, 7600 Sand Point Way NE, Seattle, WA 98115 or emailed to: Meghan Cronin meghan.f.cronin@noaa.gov',
 'area': 'Hawaii to 0N 140W and back',
 'citation': 'Please cite Zhang et al. (2019) when using this dataset. Zhang, D., M.F. Cronin, C. Meinig, J.T. Farrar, R. Jenkins, D. Peacock, J. Keene, A. Sutton, and Q. Yang. 2019. Comparing air-sea flux measure-ments from a new unmanned surface vehicle and proven platforms during the SPURS-2 field campaign. Oceanography 32(2):122–133, https://doi.org/10.5670/oceanog.2019.220.',
 'creator_institution': 'Saildrone Inc. and PMEL',
 'creator_name': 'Saildrone and PMEL',
 'creator_url': 'saildrone.com, pmel.noaa.gov',
 'funding_source': 'N

In [13]:
wmo_id_dict = {}
for k, v in reader_wmo:
    wmo_id_dict[k] = v

In [14]:
wmo_id_dict

{'1001': '4802011',
 '1002': '4802012',
 '1003': '4802013',
 '1005': '5801952',
 '1006': '5801953',
 '1020': '4803902',
 '1021': '4803903',
 '1022': '4803904',
 '1023': '4803905',
 '1024': '4803906',
 '1025': '4803907',
 '1026': '4803908',
 '1027': '4803909',
 '1028': '4803910',
 '1029': '5801954',
 '1030': '5801955',
 '1031': '1801556',
 '1032': '1801557',
 '1033': '4803911',
 '1034': '4803912',
 '1035': '4803913',
 '1036': '4803914',
 '1037': '4803915',
 '1040': '1801560',
 '1041': '4803916',
 '1042': '1801562',
 '1043': '1801563',
 '1045': '1801565',
 '1046': '1801566',
 '1048': '1801568',
 '1049': '1801569',
 '1052': '1801572',
 '1055': '1801575',
 '1057': '1801577',
 '1059': '1801579',
 '1060': '1801580',
 '1063': '1801583',
 '1064': '1801584',
 '1065': '1801585',
 '1066': '5801956',
 '1067': '5801957',
 '1068': '5801958',
 '1069': '5801959',
 '1078': '1801594',
 '1083': '1801599',
 '1084': '1801600'}

In [15]:
proj_names = {}
for k, v in reader_projnames:
    proj_names[k] = v

In [16]:
proj_names

{'1021_Atlantic': '1021 Atlantic',
 'alaska': 'Alaska',
 'antarctic_circumnavigation': 'Antarctic Circumnavigation',
 'arctic': 'Arctic',
 'arctic_single_beam': 'Arctic Single Beam',
 'atlantic_to_med': 'Atlantic to Mediterranean',
 'atomic_eurec4a': 'Atomic Eurec4a',
 'bering_afsc_pollock_survey': 'Bering AFSC Pollock Survey',
 'csiro': 'CSIRO',
 'ecmwf_atlantic_gulf_stream': 'ECMWF Atlantic Gulf Stream',
 'eurosea': 'Eurosea',
 'farallones': 'Farallones',
 'hawaiian_islands_ocean_chemistry': 'Hawaiian Islands Ocean Chemistry',
 'hurricane_monitoring': 'Hurricane Monitoring',
 'noaa_swfsc': 'NOAA SWFSC',
 'nwfsc_hake_survey': 'NWFSC Hake Survey',
 'shark': 'Shark',
 'slush': 'Slush',
 'tpos': 'TPOS',
 'usn_tf59': 'USN TF59',
 'uw_saildrone_forecasting_GTS': 'UW Saildrone Forecasting GTS',
 'west_coast_survey': 'West Coast Survey'}

In [17]:
inst_dict = {}
for k, v in reader_instnames:
    inst_dict[k] = v

In [18]:
inst_dict

{'1021_Atlantic': '',
 'alaska': 'NOAA PMEL',
 'antarctic_circumnavigation': 'NOAA PMEL',
 'arctic': 'NOAA PMEL',
 'arctic_single_beam': 'NOAA NOS',
 'atlantic_to_med': '',
 'atomic_eurec4a': '',
 'bering_afsc_pollock_survey': 'NOAA AFSC',
 'csiro': '',
 'ecmwf_atlantic_gulf_stream': 'ECMWF',
 'eurosea': '',
 'farallones': '',
 'hawaiian_islands_ocean_chemistry': 'University of Hawaii',
 'hurricane_monitoring': 'NOAA AOML PMEL',
 'noaa_swfsc': 'NOAA SWFSC',
 'nwfsc_hake_survey': 'NOAA NWFSC',
 'shark': 'Saildrone',
 'tpos': 'NOAA PMEL',
 'usn_tf59': 'US Navy',
 'uw_saildrone_forecasting_GTS': 'University of Washington',
 'west_coast_survey': 'NOAA NWFSC'}

In [19]:
res_empty = {}

### if keywords exist, run this code:

In [20]:
#to check that the titling will work
for root, dirs, files in os.walk('/home/mule-external/sci-dig/saildrone/' + mission_name + '/' + year + '/daily_files'):
    print(root)

/home/mule-external/sci-dig/saildrone/tpos/2018/daily_files
/home/mule-external/sci-dig/saildrone/tpos/2018/daily_files/1029
/home/mule-external/sci-dig/saildrone/tpos/2018/daily_files/1005
/home/mule-external/sci-dig/saildrone/tpos/2018/daily_files/1006
/home/mule-external/sci-dig/saildrone/tpos/2018/daily_files/1030


In [ ]:
tic()
directories = []

#going file by file, choosing mission and year
for root, dirs, files in os.walk('/home/mule-external/sci-dig/saildrone/' + mission_name + '/' + year + '/daily_files'):
#for root, dirs, files in os.walk('/home/mule-external/sci-dig/saildrone/' + mission_name + '/' + year + '/daily_files/1060'):
            
    for file in files:
        filenamen = os.path.join(root, file)

        #to ensure only working with netcdfs
        if file.endswith(".nc"):

            try:
    
                #prints global attributes to a string
                global_metadata = str(nco.ncks(input=filenamen, options=['-M']))

                #editing title, which changes per drone
                if root[-4] == '-':#to catch the drones with 3-number IDs
                    drid = root[-3:]
                else:
                    drid = root[-4:]
                changing_metadata['title']= str(changing_metadata['title'][:-4] + drid)
                if ('ADCP' in root) or ('adcp' in root) and ('hz' in root):
                    changing_metadata['title'] = str(inst_dict[mission_name] + ' ' + proj_names[mission_name] + ' ' + root.split('/')[6] + ' High Resolution ' + root.split('/')[-2] + ' ADCP Saildrone ' + drid)
                elif ('ADCP' in root) or ('adcp' in root):
                    changing_metadata['title'] = str(inst_dict[mission_name] + ' ' + proj_names[mission_name] + ' ' + root.split('/')[6] + ' ADCP Saildrone ' + drid)
                if ('ek80' in root) or ('EK80' in root) or ('echosounder' in root) or ('Echosounder' in root):
                    changing_metadata['title'] = str(inst_dict[mission_name] + ' ' + proj_names[mission_name] + ' ' + root.split('/')[6] + ' Echosounder Saildrone ' + drid)
                if ('wave' in root) or ('wave_spectra' in root):
                    changing_metadata['title'] = str(inst_dict[mission_name] + ' ' + proj_names[mission_name] + ' ' + root.split('/')[6] + ' Wave Data Saildrone ' + drid)
                if 'sea-trial' in root:
                    changing_metadata['title'] = str(inst_dict[mission_name] + ' ' + proj_names[mission_name] + ' ' + root.split('/')[6] + ' Sea Trial Saildrone ' + drid)
                if ('daily_files' in root) or ('real-time' in root):
                    changing_metadata['title'] = str(inst_dict[mission_name] + ' ' + proj_names[mission_name] + ' ' + root.split('/')[6] + ' NRT Saildrone ' + drid)
                if 'hz' in root:
                    changing_metadata['title'] = str(inst_dict[mission_name] + ' ' + proj_names[mission_name] + ' ' + root.split('/')[6] + ' High Resolution ' + root.split('/')[-2] + ' Saildrone ' + drid)
                if 'hz' in changing_metadata['title']:
                    changing_metadata['title'] = changing_metadata['title'].replace('hz','Hz')#this capitalizes Hz to the proper form
                
                #editing interval value
                if ('hz' in root) or ('Hz' in root):
                    changing_metadata['time_in'] = 'Hz'
                    newint = file[int(file.find('hz')) - 2]
                    if newint == '0': #if it's 10 or 20hz, add the tens digit to the zero
                        newint = file[int(file.find('hz')) - 3] + file[int(file.find('hz')) - 2]
                        changing_metadata['interval'] = newint
                    else:
                        changing_metadata['interval'] = newint
                else: #if not high resolution, then select interval based on what's in the filename
                    changing_metadata['time_in'] = 'seconds'
                    newint = file[int(file.find('min')) - 2]
                    if newint == '5': #if it's 5 that means 15 min
                        if file[int(file.find('min')) - 3] == '1':#if 15 mins
                            changing_metadata['interval'] = '900'#seconds
                        else:#if 5 mins
                            changing_metadata['interval'] = '300'#seconds        
                    else: #otherwise take it for granted to be whatever single digit it says. should be 1 for 1 min
                        changing_metadata['interval'] = '60'#seconds

                #editing time_coverage_resolution
                if (changing_metadata['interval'] == '60') and (changing_metadata['time_in'] == 'seconds'):
                    static_metadata['time_coverage_resolution'] = 'PT1M'
                elif (changing_metadata['interval'] == '300') and (changing_metadata['time_in'] == 'seconds'):
                    static_metadata['time_coverage_resolution'] = 'PT5M'
                elif (changing_metadata['interval'] == '900') and (changing_metadata['time_in'] == 'seconds'):
                    static_metadata['time_coverage_resolution'] = 'PT15M'
                elif (changing_metadata['interval'] == '1') and (changing_metadata['time_in'] == 'Hz'):
                    static_metadata['time_coverage_resolution'] = 'PT1S'
                elif (changing_metadata['interval'] == '2') and (changing_metadata['time_in'] == 'Hz'):
                    static_metadata['time_coverage_resolution'] = 'PT0.2S'
                elif (changing_metadata['interval'] == '4') and (changing_metadata['time_in'] == 'Hz'):
                    static_metadata['time_coverage_resolution'] = 'PT0.25S'
                elif (changing_metadata['interval'] == '10') and (changing_metadata['time_in'] == 'Hz'):
                    static_metadata['time_coverage_resolution'] = 'PT0.1S'
                elif (changing_metadata['interval'] == '20') and (changing_metadata['time_in'] == 'Hz'):
                    static_metadata['time_coverage_resolution'] = 'PT0.05S'

                #for loop for static global metadata attributes; overwrites all
                for attr in static_metadata:
                    #if loop to see if specific attribute exists within global metadata
                    #if there, overwrite it
                    if attr in global_metadata:
                        nco.ncatted(input=filenamen, options=['-h',
                            Atted(
                                att_name=attr,
                                var_name='global',
                                mode='o',
                                stype='c',
                                value=static_metadata[attr])])                       
                    #if not there, add it
                    else:
                        nco.ncatted(input=filenamen, options=['-h',
                            Atted(
                                att_name=attr,
                                var_name='global',
                                mode='c',
                                stype='c',
                                value=static_metadata[attr])])

                ##if loop to overwrite trajectory variable attributes to be strings and not floats
                #if (attr == 'cdm_trajectory_variables') or (attr == 'subsetVariables'):
                #    nco.ncatted(input=filenamen, options=['-h',
                #        Atted(
                #            att_name=attr,
                #           var_name='global',
                #            mode='o',
                #            stype='string',
                #            value=static_metadata[attr])]) 
                
                #if wmo_platform_code is there but wmo_id is not, rename it
                if ('wmo_platform_code' in global_metadata) and ('wmo_id' not in global_metadata):
                    nco.ncrename(input=filenamen, options=['-h',
                                            Rename("a",{'global@wmo_platform_code':'wmo_id'})])                
                
                #for loop for checking presence of changing global metadata attributes
                for attr in changing_metadata:
                    #if loop to see if specific attribute exists within global metadata
                    #if there, let it be
                    if attr in global_metadata:
                        pass
                    #if not there, add it
                    else:
                        nco.ncatted(input=filenamen, options=['-h',
                            Atted(
                                att_name=attr,
                                var_name='global',
                                mode='c',
                                stype='c',
                                value=changing_metadata[attr])])
                
                #for loop for changing incorrect specific global metadata attributes
                for attr in changing_metadata:
#                    loc_start = global_metadata.find(attr)
#                    loc_is = loc_start + len(attr) + 4
#                    loc_len = len(changing_metadata[attr])
#                    #don't check wmo for accuracy
                    if (attr == 'wmo_id') and (attr in global_metadata):
                        pass
                    elif (attr == 'wmo_id') and (attr not in global_metadata):
                        nco.ncatted(input=filenamen, options=['-h',
                            Atted(
                                att_name=attr,
                                var_name='global',
                                mode='o',
                                stype='c',
                                value=wmo_id_dict[root[-4:]])])
#                    #elif to see if specific attribute exists but is wrong, then change it
#                    elif global_metadata[loc_is:loc_is + loc_len] == changing_metadata[attr]:
#                        pass
                    else:
                        nco.ncatted(input=filenamen, options=['-h',
                            Atted(
                                att_name=attr,
                                var_name='global',
                                mode='o',
                                stype='c',
                                value=changing_metadata[attr])])
                
                #if there is a global attribute that should be removed, run the below code:
                #if 'contributor_role' in global_metadata:
                #    nco.ncatted(input=filenamen, options=['-h',
                #        Atted(
                #            att_name='contributor_role',
                #            var_name='global',
                #            mode='d')])
                        
                #checking if geospatial attributes are there and updating them if missing
                x = 0
                for gattr in geospatial_metadata:
                    #if there, let it be
                    if gattr in global_metadata:
                        pass
                    #if not there, add it
                    else:
                        fildata = xr.open_dataset(filenamen)
                        geospatial_lat_min_val = np.min(fildata.latitude).values.tolist()
                        geospatial_lat_max_val = np.max(fildata.latitude).values.tolist()
                        geospatial_lon_min_val = np.min(fildata.longitude).values.tolist()
                        geospatial_lon_max_val = np.max(fildata.longitude).values.tolist()
                        geospatial_answers_list = [geospatial_lat_max_val,geospatial_lat_min_val,geospatial_lon_max_val,geospatial_lon_min_val]
                        fildata.close()
                        nco.ncatted(input=filenamen, options=['-h',
                            Atted(
                                att_name=gattr,
                                var_name='global',
                                mode='c',
                                stype='c',
                                value=geospatial_metadata[gattr]+str('_val'))])
                    x += 1#counter to make this work in order
              
                #for loop for checking presence of unchanging untouched specific global metadata attributes
                for attr in untouched_metadata:
                    #if loop to see if specific attribute exists within global metadata
                    #if there, let it be
                    if attr in global_metadata:
                        pass
                    #if not there, add it
                    else:
                        untouched_metadata['date_created'] = datetime.datetime.now(datetime.UTC).isoformat("T") + "Z" #date created
                        untouched_metadata['drone_id'] = drid #drone id from root
                        nco.ncatted(input=filenamen, options=['-h',
                            Atted(
                                att_name=attr,
                                var_name='global',
                                mode='c',
                                stype='c',
                                value=untouched_metadata[attr])])

            #finishing out try-except, raising all exceptions
            except Exception as err:
                #save all empty files to directories and print out
                directories.append(os.path.join(root, file))
                print(err)

res_empty[mission_name] = directories
toc()

## checks

In [ ]:
root#to make sure the root is correct

In [ ]:
filenamen#to check the last filename

In [ ]:
res_empty[mission_name]#the empty files. check against the spreadsheet of empty files to make sure the amounts are the same

In [ ]:
len(res_empty[mission_name])

## changing one thing only

In [51]:
res_empty = {}

In [52]:
mission_name = 'hurricane_monitoring'#state the project
year = '2022'#state the year

In [53]:
#make new update_metadata dict for specific attribute to change
#update_metadata = {'principal_investigator' : 'names',
#                   'principal_investigator_email' : 'emails'}
#update_metadata = {'project' : 'hurricane_monitoring'}
update_metadata = {"contributor_name" : "Gregory Foltz, NOAA AOML (Co-Lead); Chidong Zhang, NOAA PMEL (Co-Lead); Andy Chiodi, NOAA PMEL/University of Washington; Calvin Mordy, NOAA PMEL/University of Washington; Catherine Edwards, Skidaway Institute of Oceanography, University of Georgia; Christian Meinig, NOAA PMEL; Dongxiao Zhang, NOAA PMEL/University of Washington; Edward (Ned) Cokelet, NOAA PMEL; Eugene Burger, NOAA PMEL; Francis Bringas, NOAA AOML; Gustavo Goni, NOAA AOML; Hristina G. Hristova, NOAA PMEL/University of Hawaii; Hyun-Sook Kim, NOAA AOML; Kathleen E. Bailey, U.S. Integrated Ocean Observing System; Kevin O'Brien, NOAA PMEL/University of Washington; Noah Lawrence-Slavas, NOAA PMEL; Joaquin Trinanes, NOAA AOML/University of Santiago, Spain; Jun Zhang, NOAA AOML/University of Miami; Lev Looney, NOAA AOML/University of Miami; Nan-Hsun Chi, NOAA PMEL; Hauke Schulz, NOAA PMEL/University of Washington; Brian D'Souza, University of Illinois at Urbana-Champaign (NOAA Lapenta Summer Intern); Colman Bashore, Middlebury College (NOAA Summer Intern); Shuichi Mori, JAMSTEC"}

In [54]:
update_metadata

{'contributor_name': "Gregory Foltz, NOAA AOML (Co-Lead); Chidong Zhang, NOAA PMEL (Co-Lead); Andy Chiodi, NOAA PMEL/University of Washington; Calvin Mordy, NOAA PMEL/University of Washington; Catherine Edwards, Skidaway Institute of Oceanography, University of Georgia; Christian Meinig, NOAA PMEL; Dongxiao Zhang, NOAA PMEL/University of Washington; Edward (Ned) Cokelet, NOAA PMEL; Eugene Burger, NOAA PMEL; Francis Bringas, NOAA AOML; Gustavo Goni, NOAA AOML; Hristina G. Hristova, NOAA PMEL/University of Hawaii; Hyun-Sook Kim, NOAA AOML; Kathleen E. Bailey, U.S. Integrated Ocean Observing System; Kevin O'Brien, NOAA PMEL/University of Washington; Noah Lawrence-Slavas, NOAA PMEL; Joaquin Trinanes, NOAA AOML/University of Santiago, Spain; Jun Zhang, NOAA AOML/University of Miami; Lev Looney, NOAA AOML/University of Miami; Nan-Hsun Chi, NOAA PMEL; Hauke Schulz, NOAA PMEL/University of Washington; Brian D'Souza, University of Illinois at Urbana-Champaign (NOAA Lapenta Summer Intern); Colma

In [55]:
#to check that the titling will work
for root, dirs, files in os.walk('/home/mule-external/sci-dig/saildrone/' + mission_name + '/' + year + '/high_res/1hz'):
    print(root)

/home/mule-external/sci-dig/saildrone/hurricane_monitoring/2022/high_res/1hz
/home/mule-external/sci-dig/saildrone/hurricane_monitoring/2022/high_res/1hz/sd-1031
/home/mule-external/sci-dig/saildrone/hurricane_monitoring/2022/high_res/1hz/sd-1032
/home/mule-external/sci-dig/saildrone/hurricane_monitoring/2022/high_res/1hz/sd-1040
/home/mule-external/sci-dig/saildrone/hurricane_monitoring/2022/high_res/1hz/sd-1059
/home/mule-external/sci-dig/saildrone/hurricane_monitoring/2022/high_res/1hz/sd-1078
/home/mule-external/sci-dig/saildrone/hurricane_monitoring/2022/high_res/1hz/sd-1083
/home/mule-external/sci-dig/saildrone/hurricane_monitoring/2022/high_res/1hz/sd-1084


In [56]:
tic()
directories = []

#going file by file, choosing mission and year
for root, dirs, files in os.walk('/home/mule-external/sci-dig/saildrone/' + mission_name + '/' + year + '/high_res/1hz'):
#for root, dirs, files in os.walk('/home/mule-external/sci-dig/saildrone/' + mission_name + '/' + year + '/daily_files/1060'):
            
    for file in files:
        filenamen = os.path.join(root, file)
        #print(filenamen)

        #to ensure only working with netcdfs
        if file.endswith(".nc"):

            try:
    
                #prints global attributes to a string
                global_metadata = str(nco.ncks(input=filenamen, options=['-M']))

                #for loop for newly updated metadata attributes; overwrites all
                for attr in update_metadata:
                    #if loop to see if specific attribute exists within global metadata
                    #if there, overwrite it
                    if attr in global_metadata:
                        nco.ncatted(input=filenamen, options=['-h',
                            Atted(
                                att_name=attr,
                                var_name='global',
                                mode='o',
                                stype='c',
                                value=update_metadata[attr])])                       
                    #if not there, add it
                    else:
                        nco.ncatted(input=filenamen, options=['-h',
                            Atted(
                                att_name=attr,
                                var_name='global',
                                mode='c',
                                stype='c',
                                value=update_metadata[attr])])

            #finishing out try-except, raising all exceptions
            except Exception as err:
                #save all empty files to directories and print out
                directories.append(os.path.join(root, file))
                print(err)

res_empty[mission_name] = directories
toc()

Elapsed time is 223.93584275245667 seconds.


### checks

In [57]:
root#to make sure the root is correct

'/home/mule-external/sci-dig/saildrone/hurricane_monitoring/2022/high_res/1hz/sd-1084'

In [58]:
filenamen#to check the last filename

'/home/mule-external/sci-dig/saildrone/hurricane_monitoring/2022/high_res/1hz/sd-1084/saildrone-gen_6-2022_atlantic_hurricane_mission_-sd1084-20220914T000000-20220914T235959-1_hz-v1.1672941953135.nc'

In [59]:
res_empty[mission_name]#the empty files. check against the spreadsheet of empty files to make sure the amounts are the same

[]

In [60]:
len(res_empty[mission_name])

0